In [1]:
# import everything for plotly dash

# !pip install dash-bootstrap-components
# !pip install plotly
# !pip install dast_html_components
# !pip install dash_core_components
# !pip install dash
# !pip install pandas
# !pip install numpy

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import json
import os
import sys
import time
import datetime


C:\Users\osama\AppData\Local\Temp\ipykernel_4748\2673932809.py:12: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\osama\AppData\Local\Temp\ipykernel_4748\2673932809.py:13: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [28]:
# read csv
df = pd.read_csv('daily.csv')
df['Day'] = pd.to_datetime(df['Day'])

# for each row read Tag col, read and remove the 'Graph' and ':' from the name
df['Tag'] = df['Tag'].apply(lambda x: x.split(':')[1] if 'Graph' in x else x)
df.head()

#remove any 'unamtched' tags
df = df[df['Tag'] != '(unmatched time)']

# group by Tag and Day
df = df.groupby(['Tag', 'Day']).sum().reset_index()
df.head()

C:\Users\osama\AppData\Local\Temp\ipykernel_4748\1178272077.py:14: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Tag,Day,Percentage
0,Game,2023-01-26,5.12
1,Game,2023-01-29,5.84
2,Game,2023-02-02,16.36
3,Game,2023-02-04,14.35
4,Game,2023-02-05,37.30


In [84]:
import pandas as pd
# read csv

def read_csv(df):
    df['Day'] = pd.to_datetime(df['Day'])

    # for each row read Tag col, read and remove the 'Graph' and ':' from the name
    df['Tag'] = df['Tag'].apply(lambda x: x.split(':')[1] if 'Graph' in x else x)
    df.head()

    #remove any 'unamtched' tags
    df = df[df['Tag'] != '(unmatched time)']
    # convert time from string to datetime then get number of hours


    df['Time'] = pd.to_timedelta(df['Time'])
    return df
# remove time column

# read csv
df = read_csv(pd.read_csv('daily.csv'))
df_a = read_csv(pd.read_csv('daily_modi.csv'))

# merge based on Day and Tag
df = pd.merge(df, df_a, how='left', on=['Day', 'Tag',])
# move values from Time_y to Time_x if not nan
# update Time and Percentage columns using combine_first
df['Time'] = df['Time_y'].combine_first(df['Time_x'])
df['Percentage'] = df['Percentage_y'].combine_first(df['Percentage_x'])
# drop unneeded columns
df = df.drop(['Time_x', 'Time_y', 'Percentage_y', 'Percentage_x'], axis=1)
# rename Percentage_x to Percentage
print(df[df['Day'] == day_to_modify])
df.tail()


          Day            Tag            Time  Percentage
7  2023-01-26       browsing 0 days 02:50:00   32.075472
8  2023-01-26         tv-web 0 days 01:10:00   13.207547
9  2023-01-26      code-work 0 days 03:58:00   44.905660
10 2023-01-26           Game 0 days 00:21:00    3.962264
11 2023-01-26          Teams 0 days 00:15:00    2.830189
12 2023-01-26        Texting 0 days 00:14:00    2.641509
13 2023-01-26  code-personal 0 days 00:02:00    0.377358


C:\Users\osama\AppData\Local\Temp\ipykernel_4748\3899427891.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Day,Tag,Time,Percentage
188,2023-02-28,Texting,0 days 00:01:00,0.22
189,2023-03-01,code-work,0 days 00:59:00,27.44
190,2023-03-01,code-personal,0 days 00:50:00,23.26
191,2023-03-01,browsing,0 days 00:50:00,23.26
192,2023-03-01,tv-web,0 days 00:34:00,15.81


In [92]:

tag_to_modify = 'code-work'
time_to_adjust = '01:00:00'
day_to_modify = pd.to_datetime('2023-03-01')

def save_modified_csv(tag_row):
    if not os.path.exists('daily_modi.csv'):
        tag_row.to_csv('daily_modi.csv', index=False)
        return
    d = read_csv(pd.read_csv('daily_modi.csv'))
    d = pd.concat([d, tag_row])
    d = d.drop_duplicates(subset=['Tag', 'Day'], keep='last')
    d.to_csv('daily_modi.csv', index=False)

def modify_time_per_tag_per_day(df,tag_to_modify,day_to_modify,time_to_adjust):
    # Find the row with the specified tag and modify the time used
    tag_row = df.loc[(df['Tag'] == tag_to_modify) & (df['Day'] == day_to_modify)]
    tag_row['Time'] = pd.to_timedelta(tag_row['Time']) + pd.to_timedelta(time_to_adjust)
    df.loc[(df['Tag'] == tag_to_modify) & (df['Day'] == day_to_modify), 'Time'] = tag_row['Time']
    # # Calculate the total time and percentage for all tags in day_to_modify
    total_time = df.loc[df['Day'] == day_to_modify]['Time'].sum()
    df.loc[df['Day'] == day_to_modify, 'Percentage'] = df.loc[df['Day'] == day_to_modify]['Time'] / total_time * 100

    #save to modified csv
    save_modified_csv(df.loc[df['Day'] == day_to_modify])

    return df

df = modify_time_per_tag_per_day(df,tag_to_modify,day_to_modify,time_to_adjust)
df

C:\Users\osama\AppData\Local\Temp\ipykernel_4748\2263653323.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Day,Tag,Time,Percentage
0,2023-01-25,browsing,0 days 01:08:00,2.042655
1,2023-01-25,Texting,0 days 00:53:00,1.592070
2,2023-01-25,code-work,2 days 04:28:00,94.562932
3,2023-01-25,code-personal,0 days 00:27:00,0.811054
4,2023-01-25,tv-web,0 days 00:20:00,0.600781
...,...,...,...,...
188,2023-02-28,Texting,0 days 00:01:00,0.220000
189,2023-03-01,code-work,0 days 06:59:00,75.768535
190,2023-03-01,code-personal,0 days 00:50:00,9.041591
191,2023-03-01,browsing,0 days 00:50:00,9.041591


In [27]:
# a pie chart
import plotly.express as px
# get entries for a day and plot a pie chart
df_ = df[df['Day'] == '2023-01-25']
print(df_)
# take pie percent from the 'Percentage' col
fig = px.pie(df_, values='Percentage', names='Tag', title='Time spent on each tag on 2023-01-26')

# names on the pie chart
fig.update_traces(textposition='inside', textinfo='percent+label')
# legend on bottom
fig.update_layout(legend=dict(

    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))


         Day            Tag            Time  Percentage     Hours
0 2023-01-25       browsing 0 days 01:08:00    9.869376  1.133333
1 2023-01-25        Texting 0 days 00:53:00    7.692308  0.883333
2 2023-01-25      code-work 0 days 08:28:00   73.730044  0.466667
3 2023-01-25  code-personal 0 days 00:27:00    3.918723  0.450000
4 2023-01-25         tv-web 0 days 00:20:00    2.902758  0.333333
5 2023-01-25          Teams 0 days 00:12:00    1.741655  0.200000
6 2023-01-25           mail 0 days 00:01:00    0.145138  0.016667


In [13]:
# heatmap of Tag vs Day of the week (Monday, Tuesday, etc)
# get the day of the week from the 'Day' col
df['Day of the week'] = df['Day'].dt.day_name()
# group by Tag and Day of the week
df_ = df.groupby(['Tag', 'Day of the week']).sum().reset_index()

# pivot the table
df_ = df_.pivot(columns='Day of the week', index='Tag', values='Percentage')
# order the columns according to the day of the week
df_ = df_[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]


# plot the heatmap
fig = go.Figure(data=go.Heatmap(
                     z=df_.values,
                        x=df_.columns,

                        y=df_.index,
                        colorscale='viridis'))
# reorder x axis
fig.update_layout(title='Time spent per tag per day of the week') 


# plot
fig.show()



C:\Users\osama\AppData\Local\Temp\ipykernel_40912\590701752.py:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [5]:
# bar chart of all records days shoing stacked Tag percentages
# group by Day
df_ = df.groupby(['Day','Tag']).sum().reset_index()
# pivot the table
df_ = df_.pivot(index='Day', columns='Tag', values='Percentage')
# plot the bar chart
fig = go.Figure(data=[go.Bar(name=col, x=df_.index, y=df_[col]) for col in df_.columns])

# stacked bats
fig.update_layout(barmode='stack')
#add x as day
fig.update_layout(xaxis_tickformat = '%d %B (%a)')
# show all x ticks
fig.update_xaxes(nticks=15)
#add title
fig.update_layout(title_text='Time spent on each tag on each day')
# plot
fig.show()

C:\Users\osama\AppData\Local\Temp\ipykernel_40912\3437487036.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [6]:
# productivity per day - bar charts 
# create col productivity - if tag in ['code-work', 'code-personal','mail','Teams','read-pdf'] then 1 else 0
df['Productivity'] = df['Tag'].apply(lambda x: 1 if x in ['code-work', 'code-personal','mail','Teams','read-pdf'] else 0)
# get sum of percentage for all productivity tags for each day
df_ = df.groupby(['Day','Tag']).sum().reset_index()
# sum of productivity tags
df_ = df_[df_['Tag'].isin(['code-work', 'code-personal','mail','Teams','read-pdf'])].groupby(['Day']).sum().reset_index()

# plot the bar chart
fig = go.Figure(data=[go.Bar(name='Percentage', x=df_['Day'], y=df_['Percentage'])])
# add x as day
fig.update_layout(xaxis_tickformat = '%d %B (%a)')
# show all x ticks
fig.update_xaxes(nticks=15)
# add title
fig.update_layout(title_text='Productivity per day')
# plot
fig.show()

C:\Users\osama\AppData\Local\Temp\ipykernel_40912\1597306120.py:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\osama\AppData\Local\Temp\ipykernel_40912\1597306120.py:7: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [7]:
# productivity per day - Line charts 
# create col productivity - if tag in ['code-work', 'code-personal','mail','Teams','read-pdf'] then 1 else 0
df['Productivity'] = df['Tag'].apply(lambda x: 1 if x in ['code-work', 'code-personal','mail','Teams','read-pdf'] else 0)
# get sum of percentage for all productivity tags for each day
df_ = df.groupby(['Day','Tag']).sum().reset_index()
# sum of productivity tags
df_ = df_[df_['Tag'].isin(['code-work', 'code-personal','mail','Teams','read-pdf'])].groupby(['Day']).sum().reset_index()

# plot the bar chart
fig = go.Figure(data=[go.Line(name='Percentage', x=df_['Day'], y=df_['Percentage'])])
# add x as day
fig.update_layout(xaxis_tickformat = '%d %B (%a)')
# show all x ticks
fig.update_xaxes(nticks=15)
# add title
fig.update_layout(title_text='Productivity per day')
# plot
fig.show()

C:\Users\osama\AppData\Local\Temp\ipykernel_40912\1827018448.py:5: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\osama\AppData\Local\Temp\ipykernel_40912\1827018448.py:7: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

c:\Users\osama\miniconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [8]:
# per minute data
# read csv
df = pd.read_csv('minute.csv')
df['Minute'] = pd.to_datetime(df['Minute'])

# for each row read Tag col, read and remove the 'Graph' and ':' from the name
df['Tag'] = df['Tag'].apply(lambda x: x.split(':')[1] if 'Graph' in x else x)
df.head()

#remove any 'unamtched' tags
df = df[df['Tag'] != '(unmatched time)']

# group by Tag and Day
df = df.groupby(['Tag', 'Minute']).sum().reset_index()
df.head()

C:\Users\osama\AppData\Local\Temp\ipykernel_40912\1566492451.py:14: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Tag,Minute,Percentage
0,Game,2023-01-26 22:34:00,100.0
1,Game,2023-01-26 22:36:00,100.0
2,Game,2023-01-26 22:37:00,100.0
3,Game,2023-01-26 22:38:00,100.0
4,Game,2023-01-26 22:39:00,100.0


In [9]:
# lets do a heatmap for a Tag vs Hour of the day
# get the hour of the day from the 'Minute' col
df['Hour of the day'] = df['Minute'].dt.hour
# group by Tag and Hour of the day
df_ = df.groupby(['Tag', 'Hour of the day']).count().reset_index()
print(df_)
# pivot the table
df_ = df_.pivot(index='Tag', columns='Hour of the day', values='Percentage')

# plot the heatmap horizontally
fig = go.Figure(data=go.Heatmap(
                        z=df_.values,
                        x=df_.columns,
                        y=df_.index,
                        colorscale='viridis'))
#title
fig.update_layout(title_text='Tag vs Hour of the day') 



# plot
fig.show()

       Tag  Hour of the day  Minute  Percentage
0     Game               18      15          15
1     Game               21      13          13
2     Game               22      43          43
3    Teams                7       7           7
4    Teams               12      14          14
..     ...              ...     ...         ...
84  tv-web               19      23          23
85  tv-web               20      86          86
86  tv-web               21      68          68
87  tv-web               22      94          94
88  tv-web               23      86          86

[89 rows x 4 columns]
